In [1]:
import pandas as pd
import os
import detect_simpsons_paradox as dsp
import numpy as np

We'll first load in some data, this has both regression and rate type trends, since this file has a weird index, we'll load it in as dataframe first

In [2]:
hit_search_rate = pd.read_csv('../data/state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI.csv',index_col='Unnamed: 0')
hit_search_rate.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,search_conducted_rate,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,0.032258,31.0,NaN,NaN,1.0,NaN,NaN,31.0
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,0.666667,2.0,1.0,0.333333,1.0,1.0,0.5,3.0
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,0.066667,15.0,NaN,NaN,1.0,NaN,NaN,15.0
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,0.111111,34.0,2.0,0.055556,2.0,2.0,0.5,36.0
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,0.049180,61.0,NaN,NaN,3.0,NaN,NaN,61.0


We can now use the labeledDataFrame with that DataFrame to create the object

In [3]:
labeled_df_setup = dsp.labeledDataFrame(hit_search_rate)

Next, we can infer the variable types

In [4]:
labeled_df_setup.infer_var_types()
labeled_df_setup.meta_df.head()

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,NaN,NaN,NaN
year,int64,ordinal,NaN,NaN,NaN
driver_gender,object,binary,NaN,NaN,NaN
driver_race,object,categorical,NaN,NaN,NaN
decriminalization,int64,binary,NaN,NaN,NaN


For this, we'll manually set these, but in the vizualization tool you can also set these with drop down menus

In [5]:
roles = {'state':['trend','groupby'], 'year':'trend', 'driver_gender':['trend','groupby'], 'driver_race':['trend','groupby'],
       'decriminalization':['groupby'], 'medical':['groupby'],
         'recreational':['groupby'], 'no_reforms':['groupby'],
       'search_conducted_false':'ignore', 'search_conducted_true':'ignore',
       'search_conducted_rate':'trend', 'contraband_found_false':'ignore',
       'contraband_found_true':'ignore', 'contraband_found_rate':'trend', 'hit_false':'ignore',
       'hit_true':'ignore', 'hit_rate':'trend', 'num_stops':'trend'}
is_count = {'state':False, 'year':False, 'driver_gender':False, 'driver_race':False,
       'decriminalization':False, 'medical':False, 'recreational':False, 'no_reforms':False,
       'search_conducted_false':True, 'search_conducted_true':True,
       'search_conducted_rate':False, 'contraband_found_false':True,
       'contraband_found_true':True, 'contraband_found_rate':False, 'hit_false':True,
       'hit_true':True, 'hit_rate':False, 'num_stops':True}
count_list = ['search_conducted_false', 'search_conducted_true','contraband_found_false',
       'contraband_found_true', 'hit_false',
       'hit_true', 'num_stops']
var_types = {'driver_gender':'categorical'}
weighting = {'hit_rate':'search_conducted_true','search_conducted_rate':'num_stops','contraband_found_rate':'num_stops'}

We'll set those next.  Above gives examples of two ways that we can specify the count values to pass them to the set_counts function, but we'll only call it once below. 

In [6]:
labeled_df_setup.set_counts(count_list)
labeled_df_setup.set_roles(roles)
labeled_df_setup.set_weighting_vars(weighting)
labeled_df_setup.set_var_types(var_types)
labeled_df_setup.meta_df

/home/smb/anaconda3/lib/python3.6/site-packages/detect_simpsons_paradox/labeled_dataframe.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.meta_df['role'][k] = v
/home/smb/anaconda3/lib/python3.6/site-packages/detect_simpsons_paradox/labeled_dataframe.py:211: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.meta_df['var_type'][k] = v


,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,trend,False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,binary,[groupby],False,NaN
medical,int64,binary,[groupby],False,NaN
recreational,int64,binary,[groupby],False,NaN
no_reforms,int64,binary,[groupby],False,NaN
search_conducted_false,float64,continuous,ignore,True,NaN


Now, we've set this up, we can also save these configurations to load them in directly in the future

In [7]:
labeled_df_setup.to_csvs('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')

We can see what this does, using a bash magic

In [8]:
%%bash
cd ../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI
ls

df.csv
meta.csv
result_df.csv


it write the three DataFrames each out to their own .csv file in that directory. If that directory exists it will overwrite without warning, if not, also creates the directory. 

Now, we can can also load the data back

In [9]:
labeled_df = dsp.labeledDataFrame('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,binary,[groupby],False,NaN
medical,int64,binary,[groupby],False,NaN
recreational,int64,binary,[groupby],False,NaN
no_reforms,int64,binary,[groupby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN


In [10]:
labeled_df.add_all_dpgmm(qual_thresh =.2)
labeled_df.df.head()

/home/smb/anaconda3/lib/python3.6/site-packages/sklearn/mixture/base.py:237: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,...,hit_true,hit_rate,num_stops,search_conducted_false_search_conducted_true_dpgmm,search_conducted_false_contraband_found_false_dpgmm,search_conducted_false_num_stops_dpgmm,search_conducted_true_contraband_found_false_dpgmm,search_conducted_true_num_stops_dpgmm,search_conducted_rate_contraband_found_false_dpgmm,contraband_found_false_num_stops_dpgmm
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,...,NaN,NaN,31.0,6,0,8,8,8,0,1
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,...,1.0,0.5,3.0,6,0,8,8,8,4,1
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,...,NaN,NaN,15.0,6,0,8,8,8,0,1
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,...,2.0,0.5,36.0,6,0,8,8,8,4,1
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,...,NaN,NaN,61.0,6,0,8,8,8,0,1


In [11]:
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,binary,[groupby],False,NaN
medical,int64,binary,[groupby],False,NaN
recreational,int64,binary,[groupby],False,NaN
no_reforms,int64,binary,[groupby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN
search_conducted_true,float64,continuous,[ignore],True,NaN


In [12]:
labeled_df.add_quantile(['hit_rate','num_stops'])

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,...,num_stops,search_conducted_false_search_conducted_true_dpgmm,search_conducted_false_contraband_found_false_dpgmm,search_conducted_false_num_stops_dpgmm,search_conducted_true_contraband_found_false_dpgmm,search_conducted_true_num_stops_dpgmm,search_conducted_rate_contraband_found_false_dpgmm,contraband_found_false_num_stops_dpgmm,hit_ratequantiles,num_stopsquantiles
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,...,31.0,6,0,8,8,8,0,1,high,low
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,...,3.0,6,0,8,8,8,4,1,high,low
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,...,15.0,6,0,8,8,8,0,1,high,low
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,...,36.0,6,0,8,8,8,4,1,high,low
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,...,61.0,6,0,8,8,8,0,1,high,low
5,AZ,2010,F,Black,0,1,0,0,224.0,8.0,...,232.0,6,0,8,8,8,0,1,high,low
6,AZ,2010,F,Hispanic,0,1,0,0,557.0,33.0,...,590.0,6,0,8,8,8,0,1,low,low
7,AZ,2010,F,Other,0,1,0,0,167.0,6.0,...,173.0,6,0,8,8,8,0,1,high,low
8,AZ,2010,F,White,0,1,0,0,3145.0,92.0,...,3237.0,6,0,8,8,8,0,1,low,mid
9,AZ,2010,M,Asian,0,1,0,0,97.0,1.0,...,98.0,6,0,8,8,8,0,1,high,low


In [13]:
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,binary,[groupby],False,NaN
medical,int64,binary,[groupby],False,NaN
recreational,int64,binary,[groupby],False,NaN
no_reforms,int64,binary,[groupby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN
search_conducted_true,float64,continuous,[ignore],True,NaN


# Using Trends

Trend objects define their name, how to compute the trend and how to choose which variables, 

extension will allow that the var lists may be passed to reduce which ones are computed

In [14]:
corrobj = dsp.all_pearson()
corrobj.get_trend_vars(labeled_df)
corrobj.regression_vars

['year',
 'search_conducted_rate',
 'contraband_found_rate',
 'hit_rate',
 'num_stops']

In [15]:
rankobj = dsp.mean_rank_trend()
linreg_obj = dsp.all_linear_trend()

# Computing Trends on a labeledDataFrame

There are two ways, we can use default setting and pass the names of the trend type or a trend object

In [16]:
labeled_df.get_subgroup_trends_1lev(['pearson_corr'])
labeled_df.result_df.head()

,feat1,feat2,subgroup_trend,subgroup_trend_quality,group_feat,subgroup,trend_type,agg_trend,agg_trend_quality
0,year,search_conducted_rate,-0.247018,0.247018,state,AZ,pearson_corr,-0.03903,0.03903
1,year,search_conducted_rate,-0.414566,0.414566,state,CO,pearson_corr,-0.03903,0.03903
2,year,search_conducted_rate,0.118238,0.118238,state,CT,pearson_corr,-0.03903,0.03903
3,year,search_conducted_rate,-0.199765,0.199765,state,IL,pearson_corr,-0.03903,0.03903
4,year,search_conducted_rate,-0.603026,0.603026,state,MA,pearson_corr,-0.03903,0.03903


Now we can use a list of objects and apply multiple trends

In [17]:
labeled_df.get_subgroup_trends_1lev([rankobj,linreg_obj])
labeled_df.result_df.sample(10)

/home/smb/anaconda3/lib/python3.6/site-packages/scipy/stats/_stats_mstats_common.py:107: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/home/smb/anaconda3/lib/python3.6/site-packages/scipy/stats/_stats_mstats_common.py:117: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/home/smb/anaconda3/lib/python3.6/site-packages/scipy/stats/_stats_mstats_common.py:119: RuntimeWarning: invalid value encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)


,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality
241,contraband_found_rate,driver_gender,search_conducted_false_search_conducted_true_d...,8,"[F, M]",0.078900,rank_trend,"[F, M]",0.17970
905,search_conducted_rate,state,search_conducted_rate_contraband_found_false_d...,5,"[NC, TX, SC, WA]",0.132800,rank_trend,"[CO, NC, VT, WI, MD, MA, CT, TX, SC, WA, RI, I...",0.38810
276,contraband_found_rate,driver_gender,search_conducted_true_contraband_found_false_d...,9,"[F, M]",0.132500,rank_trend,"[F, M]",0.17970
0,search_conducted_rate,driver_gender,state,AZ,"[F, M]",0.156000,rank_trend,"[F, M]",0.25190
739,num_stops,driver_race,driver_gender,F,"[Other, Asian, Hispanic, Black, White]",0.421000,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.49960
1252,year,search_conducted_rate,search_conducted_false_contraband_found_false_...,9,3.38173e-05,0.022502,lin_reg,-0.000353381,0.03903
834,search_conducted_rate,state,driver_race,White,"[CO, NC, VT, WI, CT, MD, MA, TX, SC, IL, WA, R...",0.506300,rank_trend,"[CO, NC, VT, WI, MD, MA, CT, TX, SC, WA, RI, I...",0.38810
272,contraband_found_rate,driver_gender,search_conducted_true_contraband_found_false_d...,5,"[F, M]",0.085800,rank_trend,"[F, M]",0.17970
424,hit_rate,driver_gender,state,VT,"[F, M]",0.228800,rank_trend,"[F, M]",0.04330
999,contraband_found_rate,state,search_conducted_rate_contraband_found_false_d...,5,"[NC, WA, TX, SC]",0.298300,rank_trend,"[NC, CO, MD, WA, CT, SC, TX, WI, IL, MA, VT, A...",0.02010


These two methods give the same, the string based version allows for simple access to default setting but passing a trend object would allow for overriding defaults and creating more custom subests of trends.

We can see what types of trends were computed from `result_df`

In [18]:
pd.unique(labeled_df.result_df['trend_type'])

array(['rank_trend', 'lin_reg'], dtype=object)

The object also stores the trend objects that have been applied, they can be used for mapping to get the distance functions that are appropriate for each trend

In [19]:
labeled_df.trend_list

In [20]:
# labeled_df.result_df['distance'] = labeled_df.result_df.apply(dist_helper,axis=1)
labeled_df.add_distance()

labeled_df.result_df.sample(10)

/home/smb/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3559: RuntimeWarning: invalid value encountered in double_scalars
  size * (size - 1) * (size - 2))


,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance
584,hit_rate,driver_race,search_conducted_true_contraband_found_false_d...,8,"[Asian, Other, Hispanic, Black, White]",0.022200,rank_trend,"[Asian, Hispanic, Other, Black, White]",0.002300,0.200000
411,contraband_found_rate,driver_race,num_stopsquantiles,high,"[White, Hispanic, Black]",0.108700,rank_trend,"[Asian, White, Other, Hispanic, Black]",0.071600,0.330000
545,hit_rate,driver_race,search_conducted_false_search_conducted_true_d...,3,"[White, Hispanic, Black]",0.413800,rank_trend,"[Asian, Hispanic, Other, Black, White]",0.002300,0.780000
1123,num_stops,state,no_reforms,0,"[RI, VT, WI, CT, MD, IL, CO, AZ, MA, NC, SC, W...",0.360400,rank_trend,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.320900,0.180000
1181,num_stops,state,search_conducted_true_num_stops_dpgmm,10,"[MD, NC, TX, CT, WI, IL, CO, WA, AZ, SC, MA]",0.098900,rank_trend,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.320900,0.710000
520,hit_rate,driver_race,state,CO,"[Other, Hispanic, Black, Asian, White]",0.494000,rank_trend,"[Asian, Hispanic, Other, Black, White]",0.002300,0.800000
260,contraband_found_rate,driver_gender,search_conducted_false_num_stops_dpgmm,5,[M],NaN,rank_trend,"[F, M]",0.179700,NaN
54,search_conducted_rate,driver_gender,search_conducted_false_num_stops_dpgmm,6,"[F, M]",0.229600,rank_trend,"[F, M]",0.251900,0.000000
652,num_stops,driver_gender,search_conducted_false_search_conducted_true_d...,5,"[F, M]",0.523800,rank_trend,"[F, M]",0.116400,0.000000
1855,search_conducted_rate,num_stops,state,TX,4.25787e+06,0.173388,lin_reg,-257201,0.065594,0.999997


Each trend object has a trend_precompute dictionary as a property that stores the intermediate values (tables of the weighted rates for ranks and correlation matrices for pearson correlation, TODO: what do we need for linreg). These can be used in vizualization.

In [21]:
labeled_df.trend_list[0].trend_precompute

{'rank_trend_agg_trend_contraband_found_rate_driver_gender': driver_gender
 F    0.003184
 M    0.006874
 dtype: float64,
 'rank_trend_agg_trend_contraband_found_rate_driver_race': driver_race
 Asian       0.002592
 White       0.004982
 Other       0.005700
 Hispanic    0.006455
 Black       0.008367
 dtype: float64,
 'rank_trend_agg_trend_contraband_found_rate_state': state
 NC    0.001062
 CO    0.002303
 MD    0.003503
 WA    0.004353
 CT    0.005716
 SC    0.006382
 TX    0.006541
 WI    0.007934
 IL    0.008215
 MA    0.008255
 VT    0.009567
 AZ    0.009980
 RI    0.013719
 dtype: float64,
 'rank_trend_agg_trend_hit_rate_driver_gender': driver_gender
 F    0.243620
 M    0.256954
 dtype: float64,
 'rank_trend_agg_trend_hit_rate_driver_race': driver_race
 Asian       0.165508
 Hispanic    0.193965
 Other       0.195967
 Black       0.269228
 White       0.279382
 dtype: float64,
 'rank_trend_agg_trend_hit_rate_state': state
 WA    0.135670
 NC    0.146822
 IL    0.205232
 AZ    0

# Filtering

In [22]:
help(labeled_df.get_trend_rows)

Help on method get_trend_rows in module detect_simpsons_paradox.ranking_processing:

get_trend_rows(feat1=None, feat2=None, group_feat=None, subgroup=None) method of detect_simpsons_paradox.labeled_dataframe.labeledDataFrame instance
    return a row of result_df based on the specified values. returned rows
    meet provided criteria for all columns (and operator) and any one of the listed
    values for each column (or operator)
    
    Parameters
    -----------
    feat1 : str, list, or  None
        trend variable name or None to include all
    feat2 : str, list, or  None
        trend variable name or None to include all
    group_feat : str, list, or  None
        groupoby variable name or None to include all
    subgroup : str, list, or  None
        value of groupby_feat or or None to include all



So, we can use that function to filter and look at subsets of the trends based on the features, groupby, or subgroups

In [23]:
labeled_df.get_trend_rows(feat1='year',subgroup=['Black','Hispanic'])

,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance
1220,year,search_conducted_rate,driver_race,Black,-0.000555503,0.039174,lin_reg,-0.000353381,0.039030,0.000129
1221,year,search_conducted_rate,driver_race,Hispanic,-0.0014828,0.177694,lin_reg,-0.000353381,0.039030,0.000719
1541,year,num_stops,driver_race,Black,-2262.72,0.169689,lin_reg,-1560.86,0.043965,0.000127
1542,year,num_stops,driver_race,Hispanic,422.667,0.014732,lin_reg,-1560.86,0.043965,0.998086


We can also filter based on SP

In [24]:
labeled_df.get_SP_rows(thresh=.2)

,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance,SP_thresh0.2
30,search_conducted_rate,driver_gender,search_conducted_false_search_conducted_true_d...,4,"[M, F]",0.881900,rank_trend,"[F, M]",0.251900,2.000000,True
44,search_conducted_rate,driver_gender,search_conducted_false_contraband_found_false_...,7,"[M, F]",0.881900,rank_trend,"[F, M]",0.251900,2.000000,True
55,search_conducted_rate,driver_gender,search_conducted_false_num_stops_dpgmm,7,"[M, F]",0.881900,rank_trend,"[F, M]",0.251900,2.000000,True
58,search_conducted_rate,driver_gender,search_conducted_false_num_stops_dpgmm,10,"[M, F]",0.178600,rank_trend,"[F, M]",0.251900,2.000000,True
63,search_conducted_rate,driver_gender,search_conducted_true_contraband_found_false_d...,3,"[M, F]",0.881900,rank_trend,"[F, M]",0.251900,2.000000,True
65,search_conducted_rate,driver_gender,search_conducted_true_contraband_found_false_d...,5,"[M, F]",0.085800,rank_trend,"[F, M]",0.251900,2.000000,True
76,search_conducted_rate,driver_gender,search_conducted_true_num_stops_dpgmm,4,"[M, F]",0.881900,rank_trend,"[F, M]",0.251900,2.000000,True
89,search_conducted_rate,driver_gender,contraband_found_false_num_stops_dpgmm,0,"[M, F]",0.379700,rank_trend,"[F, M]",0.251900,2.000000,True
107,search_conducted_rate,driver_race,state,CT,"[Asian, Other, White, Hispanic, Black]",0.443000,rank_trend,"[Asian, White, Other, Black, Hispanic]",0.277200,0.400000,True
109,search_conducted_rate,driver_race,state,MA,"[Other, Asian, White, Black, Hispanic]",0.304800,rank_trend,"[Asian, White, Other, Black, Hispanic]",0.277200,0.400000,True


# Ranking

In [28]:
labeled_df.rank_occurences_by_view(ascending=False).head(20)

Index(['feat1', 'feat2', 'group_feat', 'subgroup', 'subgroup_trend',
       'subgroup_trend_quality', 'trend_type', 'agg_trend',
       'agg_trend_quality', 'distance', 'SP_thresh0.2', 'mean_view_distance',
       'sum_view_SP_thresh0.2'],
      dtype='object')


,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance,SP_thresh0.2,mean_view_distance,sum_view_SP_thresh0.2
0,num_stops,driver_gender,search_conducted_false_search_conducted_true_d...,1,"[M, F]",0.1273,rank_trend,"[F, M]",0.1164,2.0,True,0.5,21.0
1,num_stops,driver_gender,search_conducted_false_search_conducted_true_d...,4,"[M, F]",0.4921,rank_trend,"[F, M]",0.1164,2.0,True,0.5,21.0
2,num_stops,driver_gender,search_conducted_false_search_conducted_true_d...,8,"[M, F]",0.0789,rank_trend,"[F, M]",0.1164,2.0,True,0.5,21.0
3,num_stops,driver_gender,search_conducted_false_contraband_found_false_...,5,"[M, F]",0.1628,rank_trend,"[F, M]",0.1164,2.0,True,0.5,21.0
4,num_stops,driver_gender,search_conducted_false_contraband_found_false_...,7,"[M, F]",0.4921,rank_trend,"[F, M]",0.1164,2.0,True,0.5,21.0
5,num_stops,driver_gender,search_conducted_false_num_stops_dpgmm,6,"[M, F]",0.0714,rank_trend,"[F, M]",0.1164,2.0,True,0.5,21.0
6,num_stops,driver_gender,search_conducted_false_num_stops_dpgmm,7,"[M, F]",0.4921,rank_trend,"[F, M]",0.1164,2.0,True,0.5,21.0
7,num_stops,driver_gender,search_conducted_false_num_stops_dpgmm,9,"[M, F]",0.0769,rank_trend,"[F, M]",0.1164,2.0,True,0.5,21.0
8,num_stops,driver_gender,search_conducted_true_contraband_found_false_d...,3,"[M, F]",0.4921,rank_trend,"[F, M]",0.1164,2.0,True,0.5,21.0
9,num_stops,driver_gender,search_conducted_true_contraband_found_false_d...,6,"[M, F]",0.0750,rank_trend,"[F, M]",0.1164,2.0,True,0.5,21.0


In [29]:
labeled_df.add_view_score('SP_thresh0.2',agg_type='sum',colored=False).head(10)

,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance,SP_thresh0.2,mean_view_distance,sum_view_SP_thresh0.2_x,sum_view_SP_thresh0.2_y
0,num_stops,driver_gender,search_conducted_false_search_conducted_true_d...,1,"[M, F]",0.1273,rank_trend,"[F, M]",0.1164,2.0,True,0.5,21.0,21.0
1,num_stops,driver_gender,search_conducted_false_search_conducted_true_d...,4,"[M, F]",0.4921,rank_trend,"[F, M]",0.1164,2.0,True,0.5,21.0,21.0
2,num_stops,driver_gender,search_conducted_false_search_conducted_true_d...,8,"[M, F]",0.0789,rank_trend,"[F, M]",0.1164,2.0,True,0.5,21.0,21.0
3,num_stops,driver_gender,search_conducted_false_contraband_found_false_...,5,"[M, F]",0.1628,rank_trend,"[F, M]",0.1164,2.0,True,0.5,21.0,21.0
4,num_stops,driver_gender,search_conducted_false_contraband_found_false_...,7,"[M, F]",0.4921,rank_trend,"[F, M]",0.1164,2.0,True,0.5,21.0,21.0
5,num_stops,driver_gender,search_conducted_false_num_stops_dpgmm,6,"[M, F]",0.0714,rank_trend,"[F, M]",0.1164,2.0,True,0.5,21.0,21.0
6,num_stops,driver_gender,search_conducted_false_num_stops_dpgmm,7,"[M, F]",0.4921,rank_trend,"[F, M]",0.1164,2.0,True,0.5,21.0,21.0
7,num_stops,driver_gender,search_conducted_false_num_stops_dpgmm,9,"[M, F]",0.0769,rank_trend,"[F, M]",0.1164,2.0,True,0.5,21.0,21.0
8,num_stops,driver_gender,search_conducted_true_contraband_found_false_d...,3,"[M, F]",0.4921,rank_trend,"[F, M]",0.1164,2.0,True,0.5,21.0,21.0
9,num_stops,driver_gender,search_conducted_true_contraband_found_false_d...,6,"[M, F]",0.0750,rank_trend,"[F, M]",0.1164,2.0,True,0.5,21.0,21.0


In [27]:
labeled_df.rank_occurences_by_view('sum_view_SP_thresh_0.2','SP_thresh0.2').head()

Index(['feat1', 'feat2', 'group_feat', 'subgroup', 'subgroup_trend',
       'subgroup_trend_quality', 'trend_type', 'agg_trend',
       'agg_trend_quality', 'distance', 'SP_thresh0.2', 'mean_view_distance',
       'sum_view_SP_thresh0.2'],
      dtype='object')


KeyError: 'sum_view_SP_thresh_0.2'